In [237]:
import copy

def createStateMatrix (Text):
  state = []
  j = 0

  for i in range(2):
    row = []
    row.append(Text[j])
    row.append(Text[j+2])
    state.append(row)
    j = j + 1
  
  return state


In [238]:
def shiftRow (state):
    
    shiftedState = copy.deepcopy(state)
    temp = shiftedState[0][0]
    shiftedState[0][0] = shiftedState[0][1]
    shiftedState[0][1] = temp
    shiftedText = ''
    shiftedText = shiftedText + shiftedState[0][0]
    shiftedText = shiftedText + shiftedState[1][0]
    shiftedText = shiftedText + shiftedState[0][1]
    shiftedText = shiftedText + shiftedState[1][1]
    
    return shiftedText

In [239]:
def subNibbles(Text):
  
    subText = ''

    lookupTable = {'0': 'a' , '1' : '0' , '2' : '9' , '3' : 'e' , '4' : '6' , '5' : '3' , '6' : 'f' , '7' : '5' , '8' : '1' ,
                    '9' :'d' , 'a' : 'c' , 'b' : '7' , 'c' : 'b' , 'd' : '4' , 'e' : '2' , 'f' : '8'}

    for i in Text:
      subText = subText + lookupTable[i]
     
    return subText

In [240]:
def invSubNibbles(Text):

    subText = ''

    lookupTable = {'a': '0', '0': '1', '9': '2', 'e': '3', '6': '4', '3': '5', 'f': '6', '5': '7', '1': '8',
                 'd': '9', 'c': 'a', '7': 'b', 'b': 'c', '4': 'd', '2': 'e', '8': 'f'}
    
    for i in Text:
      subText = subText + lookupTable[i]
     
    return subText

In [241]:
def generateRoundKey1(state,roundKey1):
   
   rkey1 = ''
   roundKey1[0] = hex (int(state[0][0],16) ^ int(subNibbles(state[1][1]),16) ^ 14)[2:]
   roundKey1[1] = hex (int(state[1][0],16) ^ int(roundKey1[0],16))[2:]
   roundKey1[2] = hex (int(state[0][1],16) ^ int(roundKey1[1],16))[2:]
   roundKey1[3] = hex (int(state[1][1],16) ^ int(roundKey1[2],16))[2:]
      
   return roundKey1

In [242]:
def generateRoundKey2(roundKey1,roundKey2):
   
   rkey2 = ''
   roundKey2[0] = hex (int(roundKey1[0],16) ^ int(subNibbles(roundKey1[3]),16) ^ 10)[2:]
   roundKey2[1] = hex(int(roundKey1[1],16) ^ int(roundKey2[0],16))[2:]
   roundKey2[2] = hex(int(roundKey1[2],16) ^ int(roundKey2[1],16))[2:]
   roundKey2[3] = hex(int(roundKey1[3],16) ^ int(roundKey2[2],16))[2:]

   return roundKey2

In [243]:
def stringify(roundKey):
    
    rkey =''

    for i in range(4):
       rkey = rkey + roundKey[i]

    return rkey

In [244]:
def invXOR(roundKey,shiftedCipherText):
    
    invText = hex(int(stringify(roundKey),16) ^ int(shiftedCipherText,16))[2:]

    if len(invText) == 1:
        temp = invText
        invText= '000' + temp

    elif len(invText) == 2:
        temp = invText
        invText= '00' + temp
    
    elif len(invText) == 3:
        temp = invText
        invText= '0' + temp
    
    return invText

In [245]:
def multiply(row , col):
    
    product = 0

    for i in range(2):

        a = int(row[i],16)
        b = int(col[i],16)
        m = 0
      
        while b > 0 :

          if b & 1: 
            m = m ^ a
            

          if a & 8 :
            a = (a << 1) ^ 19
           
          else:
              a = a << 1

          b = b >> 1
         
          
        product = product ^ m
        
    
    product = hex(product)
    product = product[2:]
    #print("product in hex" , product)
    return product
                

In [246]:
def mixColumns (state) :

    constantMatrix = [['9','2'],['2','9']]
    col1 = []
    col2 = []
    row1 = copy.deepcopy(constantMatrix[0])
    row2 = copy.deepcopy(constantMatrix[1])
    col1.append(state[0][0])
    col1.append(state[1][0])
    col2.append(state[0][1])
    col2.append(state[1][1])

   
    mixedText = ''
    mixedText = mixedText + multiply(row1,col1)
    mixedText = mixedText + multiply(row2,col1)
    mixedText = mixedText + multiply(row1,col2)
    mixedText = mixedText + multiply(row2,col2)
   
    return mixedText   


In [247]:

key = input("Enter Key:")
file = open(r'C:\Users\rizwan\Desktop\Manal\secret.txt' , 'r')
fileContent = file.read()
cipherText = fileContent.split()
message=''



In [248]:
def decryption(plainText):
    
   ascii_val = int(plainText,16)

   if ascii_val == 0:

      return ''
   
   else:

      return chr(ascii_val)


In [249]:
keyState = createStateMatrix(key)
roundKey1=['','','','']
roundKey2=['','','','']
roundKey1 = generateRoundKey1(keyState,roundKey1)
roundKey2 = generateRoundKey2(roundKey1,roundKey2)


for cipherBlock in cipherText:
 
  cipherTextState1 = createStateMatrix(cipherBlock)
  shiftedCipherText1 = shiftRow(cipherTextState1)
  invXORedText1 = invXOR(roundKey2,shiftedCipherText1)
  invSubText = invSubNibbles(invXORedText1)
  cipherTextState2 = createStateMatrix(invSubText)
  shiftedCipherText2 = shiftRow(cipherTextState2)
  cipherTextState3 = createStateMatrix(shiftedCipherText2)
  mixedCipherText = mixColumns(cipherTextState3)
  invXORedText2 = invXOR(roundKey1,mixedCipherText)
  plainText = invSubNibbles(invXORedText2)
  message = message + decryption(plainText[0:2]) + decryption(plainText[2:4])

In [250]:
print(message)

Gentlemen, you can't fight in here. This is the war room.
